In [ ]:
#Fazendo uma leitura dos datasets gravados na LANDING
dbutils.fs.ls("/FileStore/tables/landing")

Out[153]: []

In [ ]:
#Fazendo uma leitura dos datasets gravados na PROCESSING
dbutils.fs.ls("/FileStore/tables/processing")

Out[154]: []

In [ ]:
# Instalando a biblioteca do Kaggle
!pip install kaggle

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [ ]:
%sh
export KAGGLE_USERNAME=<USERNAME>
export KAGGLE_KEY=<KEY>
kaggle datasets download -d olistbr/brazilian-ecommerce


  0%|          | 0.00/42.6M [00:00<?, ?B/s]
 28%|██▊       | 12.0M/42.6M [00:00<00:00, 122MB/s]
 68%|██████▊   | 29.0M/42.6M [00:00<00:00, 151MB/s]
100%|██████████| 42.6M/42.6M [00:00<00:00, 160MB/s]



In [ ]:
# Listando a pasta default de download do Databricks para conferir o download do dataset em formato ZIP.
dbutils.fs.ls("file:/databricks/driver")

Out[157]: [FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096),
 FileInfo(path='file:/databricks/driver/preload_class.lst', name='preload_class.lst', size=813069),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096),
 FileInfo(path='file:/databricks/driver/metastore_db/', name='metastore_db/', size=4096),
 FileInfo(path='file:/databricks/driver/brazilian-ecommerce.zip', name='brazilian-ecommerce.zip', size=44717580)]

In [ ]:
%sh
unzip /databricks/driver/brazilian-ecommerce.zip

Archive:  /databricks/driver/brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [ ]:
# Movendo os arquivos CSV do dataset para a zona de LANDING do Datalake.
dbutils.fs.mv("file:/databricks/driver/olist_customers_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_order_reviews_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_order_items_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/product_category_name_translation.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_products_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_sellers_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_orders_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_order_payments_dataset.csv","/FileStore/tables/landing/")
dbutils.fs.mv("file:/databricks/driver/olist_geolocation_dataset.csv","/FileStore/tables/landing/")

Out[159]: True

In [ ]:
dbutils.fs.rm("file:/databricks/driver/brazilian-ecommerce.zip")

Out[160]: True

In [ ]:
#Fazendo uma leitura dos datasets gravados na LANDING
display(dbutils.fs.ls("/FileStore/tables/landing"))

path,name,size
dbfs:/FileStore/tables/landing/olist_customers_dataset.csv,olist_customers_dataset.csv,9033957
dbfs:/FileStore/tables/landing/olist_geolocation_dataset.csv,olist_geolocation_dataset.csv,61273883
dbfs:/FileStore/tables/landing/olist_order_items_dataset.csv,olist_order_items_dataset.csv,15438671
dbfs:/FileStore/tables/landing/olist_order_payments_dataset.csv,olist_order_payments_dataset.csv,5777138
dbfs:/FileStore/tables/landing/olist_order_reviews_dataset.csv,olist_order_reviews_dataset.csv,14451670
dbfs:/FileStore/tables/landing/olist_orders_dataset.csv,olist_orders_dataset.csv,17654914
dbfs:/FileStore/tables/landing/olist_products_dataset.csv,olist_products_dataset.csv,2379446
dbfs:/FileStore/tables/landing/olist_sellers_dataset.csv,olist_sellers_dataset.csv,174703
dbfs:/FileStore/tables/landing/product_category_name_translation.csv,product_category_name_translation.csv,2613


In [ ]:
# Definindo os arquivos que serao carregados da LANDING

arq_orders = "dbfs:/FileStore/tables/landing/olist_orders_dataset.csv"
arq_customers = "dbfs:/FileStore/tables/landing/olist_customers_dataset.csv"
arq_order_items = "dbfs:/FileStore/tables/landing/olist_order_items_dataset.csv"
arq_sellers = "dbfs:/FileStore/tables/landing/olist_sellers_dataset.csv"
arq_products = "dbfs:/FileStore/tables/landing/olist_products_dataset.csv"
arq_reviews = "dbfs:/FileStore/tables/landing/olist_order_reviews_dataset.csv"
arq_order_payments = "dbfs:/FileStore/tables/landing/olist_order_payments_dataset.csv"

In [ ]:
# Importando as bibliotecas para definicao de schema dos dados

from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, DateType, DoubleType, TimestampType

In [ ]:
# Definindo os schemas dos datasets individualmente

schema_df_orders = StructType([
    StructField("order_id", StringType()),
    StructField("customer_id", StringType()),
    StructField("order_status", StringType()),
    StructField("order_purchase_timestamp", TimestampType()),
    StructField("order_approved_at", DateType()),
    StructField("order_delivered_carrier_date", DateType()),
    StructField("order_delivered_customer_date", DateType()),
    StructField("order_estimated_delivery_date", DateType())
])

schema_df_customers = StructType([
    StructField("customer_id", StringType()),
    StructField("customer_unique_id", StringType()),
    StructField("customer_zip_code_prefix", StringType()),
    StructField("customer_city", StringType()),
    StructField("customer_state", StringType())
])

schema_df_order_items = StructType([
    StructField("order_id", StringType()),
    StructField("order_item_id", StringType()),
    StructField("product_id", StringType()),
    StructField("seller_id", StringType()),
    StructField("shipping_limit_date", DateType()),
    StructField("price", FloatType()),
    StructField("freight_value", FloatType())
])

schema_df_sellers = StructType([
    StructField("seller_id", StringType()),
    StructField("seller_zip_code_prefix", StringType()),
    StructField("seller_city", StringType()),
    StructField("seller_state", StringType())
])

schema_df_products = StructType([
    StructField("product_id", StringType()),
    StructField("product_category_name", StringType()),
    StructField("product_name_lenght", IntegerType()),
    StructField("product_description_lenght", IntegerType()),
    StructField("product_photos_qty", IntegerType()),
    StructField("product_weight_g", FloatType()),
    StructField("product_length_cm", FloatType()),
    StructField("product_height_cm", FloatType()),
    StructField("product_width_cm", FloatType())
])

schema_df_reviews = StructType([
    StructField("review_id", StringType()),
    StructField("order_id", StringType()),
    StructField("review_score", IntegerType()),
    StructField("review_comment_title", StringType()),
    StructField("review_comment_message", StringType()),
    StructField("review_creation_date", DateType()),
    StructField("review_answer_timestamp", DateType())
])

schema_df_payments = StructType([
    StructField("order_id", StringType()),
    StructField("payment_sequential", IntegerType()),
    StructField("payment_type", StringType()),
    StructField("payment_installments", IntegerType()),
    StructField("payment_value", FloatType())
])

In [ ]:
# Carregando os datasets

df_orders = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_orders)\
.load(arq_orders)

df_customers = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_customers)\
.load(arq_customers)

df_order_items = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_order_items)\
.load(arq_order_items)

df_sellers = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_sellers)\
.load(arq_sellers)

df_products = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_products)\
.load(arq_products)

df_reviews = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_reviews)\
.load(arq_reviews)

df_payments = spark.read.format("csv")\
.option("header", "True")\
.schema(schema_df_payments)\
.load(arq_order_payments)


In [ ]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: date (nullable = true)
 |-- order_delivered_carrier_date: date (nullable = true)
 |-- order_delivered_customer_date: date (nullable = true)
 |-- order_estimated_delivery_date: date (nullable = true)



In [ ]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [ ]:
df_order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: date (nullable = true)
 |-- price: float (nullable = true)
 |-- freight_value: float (nullable = true)



In [ ]:
df_sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [ ]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: float (nullable = true)
 |-- product_length_cm: float (nullable = true)
 |-- product_height_cm: float (nullable = true)
 |-- product_width_cm: float (nullable = true)



In [ ]:
df_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: date (nullable = true)
 |-- review_answer_timestamp: date (nullable = true)



In [ ]:
df_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: float (nullable = true)



In [ ]:
df_orders.show()

+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|       2017-10-02|                  2017-10-04|                   2017-10-10|                   2017-10-18|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|       2018-07-26|                  2018-07-26|                   2018-08-07|                   2018-08-13|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered

In [ ]:
df_customers.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                   08775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [ ]:
df_order_items.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|         2017-09-19|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|         2017-05-03| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|         2018-01-18| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|         2018-08-15| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|         2017-02-13| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [ ]:
df_sellers.show()

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                 04195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|768a86e36ad6aae3d...|                 01529|        sao paulo|          SP|
|ccc4bbb5f32a6ab2b...|                 80310|         curitiba|          PR|

In [ ]:
df_products.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|           225.0|             16.0|             10.0|            14.0|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|          1000.0|             30.0|             18.0|            20.0|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [ ]:
df_reviews.show()

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null|          2018-01-18|             2018-01-18|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null|          2018-03-10|             2018-03-11|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null|          2018-02-17|             2018-02-18|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem antes ...|          2017-04-21|   

In [ ]:
df_payments.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [ ]:
# Converte para formato parquet
df_orders.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_orders.parquet")

df_customers.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_customers.parquet")

df_order_items.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_order_items.parquet")

df_sellers.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_sellers.parquet")

df_products.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_products.parquet")

df_reviews.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_reviews.parquet")

df_payments.write.format("parquet")\
.mode("overwrite")\
.save("/FileStore/tables/processing/df_payments.parquet")


In [ ]:
#Fazendo uma leitura dos datasets gravados na LANDING
dbutils.fs.ls("/FileStore/tables/landing")

Out[181]: [FileInfo(path='dbfs:/FileStore/tables/landing/olist_customers_dataset.csv', name='olist_customers_dataset.csv', size=9033957),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_geolocation_dataset.csv', name='olist_geolocation_dataset.csv', size=61273883),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_order_items_dataset.csv', name='olist_order_items_dataset.csv', size=15438671),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_order_payments_dataset.csv', name='olist_order_payments_dataset.csv', size=5777138),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_order_reviews_dataset.csv', name='olist_order_reviews_dataset.csv', size=14451670),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_orders_dataset.csv', name='olist_orders_dataset.csv', size=17654914),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_products_dataset.csv', name='olist_products_dataset.csv', size=2379446),
 FileInfo(path='dbfs:/FileStore/tables/landing/olist_sellers_dataset.csv

In [ ]:
#Fazendo uma leitura dos datasets gravados na PROCESSING
dbutils.fs.ls("/FileStore/tables/processing")

Out[182]: [FileInfo(path='dbfs:/FileStore/tables/processing/df_customers.parquet/', name='df_customers.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_order_items.parquet/', name='df_order_items.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_orders.parquet/', name='df_orders.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_payments.parquet/', name='df_payments.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_products.parquet/', name='df_products.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_reviews.parquet/', name='df_reviews.parquet/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/processing/df_sellers.parquet/', name='df_sellers.parquet/', size=0)]